# Case - Modelo Preditivo para aprovacao de credito

## 00 - Pré-processamento dos Dados
<p>
Modelos de score de crédito calculam a probabilidade de inadimplência e são uma das
principais ferramentas utilizadas por diversas empresas para aprovar ou negar um crédito.
O objetivo deste desafio é criar um modelo preditivo calculando a probabilidade de
inadimplência de cada novo pedido de crédito.
    
    
<b>Dados</b>:
Cada linha representa um cliente e as colunas representam os dados (informações) desses
clientes.
A variável resposta é a coluna inadimplente, que indica se o tomador veio a se tornar
inadimplente(1) ou não(0).
    
    
<b>As variáveis da base de dados são descritas abaixo:</b>
    <ul>
        <li>idade: A idade do cliente</li>
        <li>numero_de_dependentes: O número de pessoas dependentes do cliente.</li>
        <li>salario_mensal: Salário mensal do cliente.</li>
        <li>numero_emprestimos_imobiliarios: Quantidade de empréstimos imobiliários que o
cliente possui em aberto.</li>
        <li>util_linhas_inseguras: Quanto que o cliente está usando, relativamente ao limite
dele, de linhas de crédito que não são seguradas por qualquer bem do tomador
e.g: imoveis, carros etc.</li>
        <li>vezes_passou_de_30_59_dias: Número de vezes que o cliente atrasou, entre 30
e 59 dias, o pagamento de um empréstimo.</li>
        <li>razao_debito: Razão entre as dívidas e o patrimônio do tomador. razão débito =
Dividas/Patrimônio</li>
        <li>numero_linhas_crdto_aberto: Número de empréstimos em aberto pelo cliente.</li>
        <li>numero_de_vezes_que_passou_60_89_dias: Número de vezes que o cliente
atrasou, entre 60 e 89 dias, o pagamento de um empréstimo.</li>        
    </ul>
    
Obs: Estes dados foram retirados de terceiros, portanto é possível que existam
incoerências, o que é perfeitamente comum em dados reais.


<b>Objetivo:</b>
Construir qualquer modelo preditivo utilizando o arquivo treino.csv.
Utilize este modelo para gerar as previsões na base teste.csv, inserindo uma nova coluna
na tabela de dados do arquivo teste.csv que contenha as previsões e nomeie esta coluna
com o nome "inadimplente".
Espera-se que não seja necessário mais que um dia para a resolução deste problema.

</p>

<p>
    <b>Author: </b>Joao Paulo Ribeiro dos Santos<br>
    <b>E-mail: </b>joao1615@gmail.com  <b>ou</b>  joao1615@faculdadefia.edu.br<br>
    <b>Telefone: </b> (11)9 6816-5227
</p>

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Joao Paulo" --iversions

Author: Joao Paulo

pandas: 1.2.1
numpy : 1.19.2



In [3]:
# Importacao dos datasets
df_train = pd.read_csv('case_datarisk/treino.csv')
df_test = pd.read_csv('case_datarisk/teste.csv')

In [4]:
# Dimensao dos dados
print('Base de treino possui {} linhas e {} colunas'.format(df_train.shape[0], df_train.shape[1]))
print('Base de teste possui {} linhas e {} colunas'.format(df_test.shape[0], df_test.shape[1]))

Base de treino possui 110000 linhas e 11 colunas
Base de teste possui 40000 linhas e 10 colunas


In [5]:
# Tipo dos dados
df_train.dtypes

inadimplente                               int64
util_linhas_inseguras                    float64
idade                                      int64
vezes_passou_de_30_59_dias                 int64
razao_debito                             float64
salario_mensal                           float64
numero_linhas_crdto_aberto                 int64
numero_vezes_passou_90_dias                int64
numero_emprestimos_imobiliarios            int64
numero_de_vezes_que_passou_60_89_dias      int64
numero_de_dependentes                    float64
dtype: object

In [6]:
# Tipo dos dados
df_test.dtypes

util_linhas_inseguras                    float64
idade                                      int64
vezes_passou_de_30_59_dias                 int64
razao_debito                             float64
salario_mensal                           float64
numero_linhas_crdto_aberto                 int64
numero_vezes_passou_90_dias                int64
numero_emprestimos_imobiliarios            int64
numero_de_vezes_que_passou_60_89_dias      int64
numero_de_dependentes                    float64
dtype: object

In [7]:
# Verificar as primeiras linhas
# Movimentar a coluna target para o final do dataframe
df_train = df_train[[c for c in df_train if c not in ['inadimplente']] + ['inadimplente']]

df_train.head(5)

,util_linhas_inseguras,idade,vezes_passou_de_30_59_dias,razao_debito,salario_mensal,numero_linhas_crdto_aberto,numero_vezes_passou_90_dias,numero_emprestimos_imobiliarios,numero_de_vezes_que_passou_60_89_dias,numero_de_dependentes,inadimplente
0,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0,1
1,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0,0
2,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0,0
3,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0,0
4,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0,0


In [8]:
# Verificar as primeiras linhas
df_test.head(5)

,util_linhas_inseguras,idade,vezes_passou_de_30_59_dias,razao_debito,salario_mensal,numero_linhas_crdto_aberto,numero_vezes_passou_90_dias,numero_emprestimos_imobiliarios,numero_de_vezes_que_passou_60_89_dias,numero_de_dependentes
0,0.025849,62,0,0.081775,8180.0,3,0,2,0,0.0
1,0.667083,55,0,0.153112,2200.0,3,0,0,0,0.0
2,0.007093,44,0,0.148800,7499.0,20,0,1,0,0.0
3,0.091213,54,0,0.351635,5900.0,15,0,1,1,0.0
4,0.112680,54,0,0.065959,2167.0,3,0,0,0,0.0


In [9]:
df_train.columns

Index(['util_linhas_inseguras', 'idade', 'vezes_passou_de_30_59_dias',
       'razao_debito', 'salario_mensal', 'numero_linhas_crdto_aberto',
       'numero_vezes_passou_90_dias', 'numero_emprestimos_imobiliarios',
       'numero_de_vezes_que_passou_60_89_dias', 'numero_de_dependentes',
       'inadimplente'],
      dtype='object')

In [10]:
df_train.dtypes

util_linhas_inseguras                    float64
idade                                      int64
vezes_passou_de_30_59_dias                 int64
razao_debito                             float64
salario_mensal                           float64
numero_linhas_crdto_aberto                 int64
numero_vezes_passou_90_dias                int64
numero_emprestimos_imobiliarios            int64
numero_de_vezes_que_passou_60_89_dias      int64
numero_de_dependentes                    float64
inadimplente                               int64
dtype: object

### Classificacao das Variaveis

In [11]:
# Classificando as variaveis 'df_train'
df_dict = pd.DataFrame(df_train.columns, columns = ['Variaveis'])


df_dict['Classificacao Variavel'] = \
                np.where(np.isin(df_dict['Variaveis'],['idade']),'Qualitativa Nominal',
                         np.where(np.isin(df_dict['Variaveis'],
                                          ['salario_mensal', 'razao_debito', 'util_linhas_inseguras']),
                                 'Quantitativa Continua', 
                                 'Quantitativa Discreta'))

df_dict

,Variaveis,Classificacao Variavel
0,util_linhas_inseguras,Quantitativa Continua
1,idade,Qualitativa Nominal
2,vezes_passou_de_30_59_dias,Quantitativa Discreta
3,razao_debito,Quantitativa Continua
4,salario_mensal,Quantitativa Continua
5,numero_linhas_crdto_aberto,Quantitativa Discreta
6,numero_vezes_passou_90_dias,Quantitativa Discreta
7,numero_emprestimos_imobiliarios,Quantitativa Discreta
8,numero_de_vezes_que_passou_60_89_dias,Quantitativa Discreta
9,numero_de_dependentes,Quantitativa Discreta


In [12]:
# Classificando as variaveis 'df_test'
df_dict = pd.DataFrame(df_test.columns, columns = ['Variaveis'])


df_dict['Classificacao Variavel'] = \
                np.where(np.isin(df_dict['Variaveis'],['idade']),'Qualitativa Nominal',
                         np.where(np.isin(df_dict['Variaveis'],
                                          ['salario_mensal', 'razao_debito', 'util_linhas_inseguras']),
                                 'Quantitativa Continua', 
                                 'Quantitativa Discreta'))

df_dict

,Variaveis,Classificacao Variavel
0,util_linhas_inseguras,Quantitativa Continua
1,idade,Qualitativa Nominal
2,vezes_passou_de_30_59_dias,Quantitativa Discreta
3,razao_debito,Quantitativa Continua
4,salario_mensal,Quantitativa Continua
5,numero_linhas_crdto_aberto,Quantitativa Discreta
6,numero_vezes_passou_90_dias,Quantitativa Discreta
7,numero_emprestimos_imobiliarios,Quantitativa Discreta
8,numero_de_vezes_que_passou_60_89_dias,Quantitativa Discreta
9,numero_de_dependentes,Quantitativa Discreta


### Verificar Valores Nulos

In [13]:
## Quantidade de valores nulos por coluna
df_train.isnull().sum()

util_linhas_inseguras                        0
idade                                        0
vezes_passou_de_30_59_dias                   0
razao_debito                                 0
salario_mensal                           21763
numero_linhas_crdto_aberto                   0
numero_vezes_passou_90_dias                  0
numero_emprestimos_imobiliarios              0
numero_de_vezes_que_passou_60_89_dias        0
numero_de_dependentes                     2878
inadimplente                                 0
dtype: int64

In [14]:
## Porcentagem de valores nulos por coluna
df_train.isnull().sum() / len(df_train) * 100

util_linhas_inseguras                     0.000000
idade                                     0.000000
vezes_passou_de_30_59_dias                0.000000
razao_debito                              0.000000
salario_mensal                           19.784545
numero_linhas_crdto_aberto                0.000000
numero_vezes_passou_90_dias               0.000000
numero_emprestimos_imobiliarios           0.000000
numero_de_vezes_que_passou_60_89_dias     0.000000
numero_de_dependentes                     2.616364
inadimplente                              0.000000
dtype: float64

In [15]:
# Dataframe com os dados Nulos
df_train[df_train.isna().any(axis=1)]

,util_linhas_inseguras,idade,vezes_passou_de_30_59_dias,razao_debito,salario_mensal,numero_linhas_crdto_aberto,numero_vezes_passou_90_dias,numero_emprestimos_imobiliarios,numero_de_vezes_que_passou_60_89_dias,numero_de_dependentes,inadimplente
6,0.305682,57,0,5710.0,NaN,8,0,3,0,0.0,0
8,0.116951,27,0,46.0,NaN,2,0,0,0,NaN,0
16,0.061086,78,0,2058.0,NaN,10,0,2,0,0.0,0
32,0.083418,62,0,977.0,NaN,6,0,1,0,0.0,0
41,0.072898,81,0,75.0,NaN,7,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
109980,1.000000,80,0,35.0,NaN,0,0,0,0,0.0,0
109986,0.938906,84,0,5734.0,NaN,5,0,1,0,0.0,0
109990,0.535453,60,0,1619.0,NaN,7,0,1,0,0.0,0
109993,0.102095,59,0,2389.0,NaN,6,0,1,0,0.0,0


In [16]:
# Verificar se existem salarios zerados
df_train[df_train['salario_mensal'] == 0].count()[0]

1216

In [17]:
# Verificar se existem salarios com valor menor que zero
df_train[df_train['salario_mensal'] < 0].count()[0]

0

In [18]:
# Verificar se existem salarios zerados
df_train.loc[df_train['salario_mensal'] <= 0]

,util_linhas_inseguras,idade,vezes_passou_de_30_59_dias,razao_debito,salario_mensal,numero_linhas_crdto_aberto,numero_vezes_passou_90_dias,numero_emprestimos_imobiliarios,numero_de_vezes_que_passou_60_89_dias,numero_de_dependentes,inadimplente
14,0.019657,76,0,477.0,0.0,6,0,1,0,0.0,0
50,0.818978,73,0,3095.0,0.0,9,0,1,1,0.0,0
73,0.059669,31,0,3162.0,0.0,11,0,2,0,1.0,0
90,0.039388,51,0,15466.0,0.0,7,0,0,0,0.0,0
298,0.085152,25,0,1005.0,0.0,5,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
108994,0.001706,74,0,15.0,0.0,10,0,0,0,0.0,0
109207,0.676427,46,0,2527.0,0.0,13,0,1,0,0.0,0
109243,0.020704,39,0,1403.0,0.0,3,0,1,0,0.0,0
109510,0.240635,55,0,54.0,0.0,3,0,0,0,0.0,0


É possível identificar que a coluna 'numero_de_dependentes' possui aproximadamente 2,6% de dados nulos, sendo assim deletaremos esses registros. Uma outra coluna que possui dados nulos é a 'salario_mensal' e como ela possui quase 20% de dados nulos, que precisaremos substituir esses valores pela média ou mediana, pra isso iremos analisar essas metricas antes.

In [19]:
df_train['salario_mensal'].describe()

count    8.823700e+04
mean     6.637411e+03
std      1.338395e+04
min      0.000000e+00
25%      3.400000e+03
50%      5.400000e+03
75%      8.225000e+03
max      3.008750e+06
Name: salario_mensal, dtype: float64

Os dados estão muito dispersos, onde há registro da salarios com valor 0 e alguns que chegam a alguns milhoes. Diante desses extremos será aplicada a mediada aos registros nulos, e excluiremos os registros nulos que correpondem ao numero de dependentes dado que representam cerca de 1% dos dados totais

In [20]:
# Tratar os dados nulos
df_train_treated = df_train.dropna(subset=['numero_de_dependentes'], how='all')

df_train_treated = df_train_treated.fillna({'salario_mensal': df_train_treated['salario_mensal'].median()})

In [21]:
# Verificar se ainda existem valores nulos
df_train_treated.isnull().sum()

util_linhas_inseguras                    0
idade                                    0
vezes_passou_de_30_59_dias               0
razao_debito                             0
salario_mensal                           0
numero_linhas_crdto_aberto               0
numero_vezes_passou_90_dias              0
numero_emprestimos_imobiliarios          0
numero_de_vezes_que_passou_60_89_dias    0
numero_de_dependentes                    0
inadimplente                             0
dtype: int64

In [22]:
# Verificar se o dataframe correspondente aos testes também possui registros nulos
df_test.isnull().sum()

util_linhas_inseguras                       0
idade                                       0
vezes_passou_de_30_59_dias                  0
razao_debito                                0
salario_mensal                           7968
numero_linhas_crdto_aberto                  0
numero_vezes_passou_90_dias                 0
numero_emprestimos_imobiliarios             0
numero_de_vezes_que_passou_60_89_dias       0
numero_de_dependentes                    1046
dtype: int64

In [23]:
# Tratar os dados nulos
df_test_treated = df_test.fillna({'numero_de_dependentes': 0})
df_test_treated = df_test_treated.fillna({'salario_mensal': df_test_treated['salario_mensal'].median()})

In [24]:
# Verificar se ainda existem valores nulos
df_test_treated.isnull().sum()

util_linhas_inseguras                    0
idade                                    0
vezes_passou_de_30_59_dias               0
razao_debito                             0
salario_mensal                           0
numero_linhas_crdto_aberto               0
numero_vezes_passou_90_dias              0
numero_emprestimos_imobiliarios          0
numero_de_vezes_que_passou_60_89_dias    0
numero_de_dependentes                    0
dtype: int64

### Dados Divergentes

In [25]:
# Verificar existem valores negativos em alguma coluna
(df_train_treated < 0).any().any()

False

In [26]:
# Verificar existem valores negativos em alguma coluna
(df_test_treated < 0).any().any()

False

### Exportando os dados

In [27]:
df_train_treated.to_csv('processed_data/train_treated.csv', index = False)

In [28]:
df_test_treated.to_csv('processed_data/test_treated.csv', index = False)